In [104]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython

In [105]:
plt.rcParams["figure.figsize"] = (14,4)

In [106]:
# using time module
import time
  
# ts stores the time in seconds
ts = time.time()
  
# print the current timestamp
print(ts)

1655500581.3547916


##Creating Time stamp info

In [79]:
def time_int(ts):
    Time = ts%(100) #Coding upto 100s to limit navigation message to 50 bits
    Time = int(Time*(10**12)) #Precision upto picoseconds
    #print(Time)
    return Time

In [89]:
def TimeStamp(Time):
    TimeBin = bin(Time).replace("0b", "")

    def Convert(string):
        list1=[]
        list1[:0]=string
        return list1
    Tlist = Convert(TimeBin)
    Tlist = list(map(int, Tlist))

    for i in range (50 - len(Tlist)): #Padding zeroes
        Tlist.insert(0,0)

    for i in range(50):
        if Tlist[i]==0:
            Tlist[i]=-1
    #plt.stem(Tlist)
    return(Tlist) #Time stamp generated


##Gold Sequence

In [111]:
class LFSR:

    def __init__(self, taps, init_register, output=None, samples_per_frame=None, matlab=False):
        if taps[-1] != 0:
            raise AttributeError("The first and last taps must be connected.")
        if taps[0] != len(init_register):
            raise AttributeError("The first and last taps must be connected.")
        if output is None:
            output = [len(init_register)]
        if any([((i > len(init_register)) | (i < 1)) for i in output]):
            raise AttributeError("The output tap(s) must exist!")

        if samples_per_frame is None:
            self.samples = 2**len(init_register) - 1
        else:
            self.samples = samples_per_frame

        self.init_register = init_register.copy()
        self.register = init_register.copy()
        self.output = output
        if matlab:
            self.taps = [taps[0] - tap for tap in reversed(taps)]
            self.taps = self.taps[:-1]
        else:
            self.taps = taps[:-1]
        self.cycled_through = False

    def reset(self):
        self.register = self.init_register.copy()

    def step(self):
        frame = []
        for _ in range(self.samples):
            frame.append(self.shift())
        return frame

    def shift(self):
        out = [self.register[i-1] for i in self.output]
        if len(out) > 1:
            out = sum(out) % 2
        else:
            out = out[0]

        feedback = sum([self.register[i-1] for i in self.taps]) % 2
        for i in reversed(range(len(self.register) - 1)):
            self.register[i+1] = self.register[i]

        self.register[0] = feedback
        if self.register == self.init_register:
            self.cycled_through = True
        return out


class Gold:

    def __init__(self, first_polynomial, first_initial_conditions,
                 second_polynomial, second_initial_conditions,
                 samples_per_frame=None, index=0, matlab=False, debug=False):
        self.mls1 = LFSR(first_polynomial, first_initial_conditions, samples_per_frame=samples_per_frame, matlab=matlab)
        self.mls2 = LFSR(second_polynomial, second_initial_conditions, samples_per_frame=samples_per_frame, matlab=matlab)
        self.index = index
        self.cycled_through = False
        self.debug = debug

    def step(self):
        u = self.mls1.step()
        v = np.roll(self.mls2.step(), -self.index).tolist()
        g = (np.logical_xor(u, v) * 1).tolist()
        if self.debug:
            print("u = {}\nv = {}\nG = {}".format(u, v, g))
        if self.mls1.cycled_through or self.mls2.cycled_through:
            self.cycled_through = True
        return g

    def reset(self):
        self.mls1.reset()
        self.mls2.reset()


if __name__ == "__main__":

    # Gold sequence
    init = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

    poly1 = [10, 6, 5, 3, 0]
    poly2 = [10, 6, 5, 2, 0]
    frame_length = 2**len(init) - 1     # Maximum length before the code repeats itself.

    gold = Gold(poly1, init, poly2, init, matlab=True)
    gold_sequence = gold.step()

##Creating PRN

In [112]:
def PRN(gold_sequence, b):
    c = []
    for i in range(20):
        c+=gold_sequence
    prn = []
    for i in range(50):
        if b[i] == 1:
            prn+= c
        else:
            prn += np.negative(c).tolist()
    return prn

##Creating GPS signal
###To create transmitted signal, multiply above sequence with cosine of freq. 1575.42 MHz. For simulation purposes, this step is unnecessary.


In [86]:
def GPS_Signal(Time,gold_sequence):
    b = TimeStamp(Time)
    gps_signal = PRN(gold_sequence, b)
    return gps_signal

##GPS detector
###Assuming the detector is placed at a distance of 20,200 Km, i.e 0.0673 light seconds which is the origin of our coordinate system

In [87]:
def time_detector(Time):
    Time_d = Time + 67333333333
    return Time_d

In [102]:
def Detector(gps_signal, Time):
    gps_tlist = []
    for i in range(50):
        for x in range(20460*i, 20460*(i+1)):
            if(gps_signal[x] ==1):
                gps_tlist.append(1)
                break
            if(gps_signal[x]==-1):
                gps_tlist.append(0)
                break
    #plt.stem(gps_tlist)

    s = [str(i) for i in gps_tlist]
    res = str("".join(s))
    signal_time = int(res,2)
    
    #print(signal_time)

    distance = (((Time - signal_time)*3/(10**7)) - 20200)*1000
    return(distance)


##Testing Detector function

In [107]:
Time = time_int(ts)
gps_signal = GPS_Signal(Time,gold_sequence)
Time_d = time_detector(Time)


dist = Detector(gps_signal,Time_d)
print(dist)  #Minimal error due to conversion from float to int, precise upto millimeter range

-0.000100000761449337


##Spoofer 
### Given a co-ordinate, spoofer should be able to create a GPS signal with desired time stamp

In [100]:
def Spoofer(dist):
    Time = time_int(ts)
    Time = Time - 67333333333 - (dist/3)*(10**4) #Subtracting the propagation time from Earth to satellite and the time taken to cover extra distance
    gps_signal = GPS_Signal(int(Time),gold_sequence)

    return(gps_signal)

In [110]:
Spoofed_dist = 50   #Meters
gps_signal_spoofed = Spoofer(Spoofed_dist)

dist = Detector(gps_signal_spoofed,Time)
print(dist) #Fairly accurate 

49.999999999272404
